In [166]:
import pandas as pdf
from tqdm import tqdm

In [184]:
# study dates (from here: https://ftp.ncbi.nlm.nih.gov/dbgap/studies/)
dates = pd.read_csv('studies_dates.tsv',sep='\t',names=['study','Last modified','time','size','-'])
dict_dates = pd.Series(dates['Last modified'].values,index=dates.study).to_dict()
print(len(dict_dates.keys()))

1707


In [185]:
# retrieve study comments from Ftp_Table_of_Contents.xml (from here: https://ftp.ncbi.nlm.nih.gov/dbgap/studies/)
df = pd.read_csv('var_report.comments.tsv',sep='\t',names=['file','link','comment'])
df['study'] = df['file'].str.split('\.',expand=True)[0]

dates = pd.read_csv('studies_dates.txt',sep=' ',names=['study','Last modified'])
dict_dates = pd.Series(dates['Last modified'].values,index=dates.study).to_dict()
df['Last modified'] = df['study'].map(dict_dates)

df['Last modified'] = pd.to_datetime(df['Last modified'])
df = df.sort_values(by='Last modified')
df = df[~pd.isna(df['Last modified'])] # some studies are not reported in https://ftp.ncbi.nlm.nih.gov/dbgap/studies/

# select studies release from 2018
df = df[(df['Last modified'] > '2018-01-01')]
df['version'] = df['file'].str.split('\.',expand=True)[1].str.split('v',expand=True)[1]
df = df[df['version'].isin(df['version'].unique()[df['version'].unique()!=None])] # two files have 'None' version
df['version'] = df['version'].astype(int)

# keep only the most recent version of each study
a = df.groupby('study')['version'].max().reset_index()
a_dict = pd.Series(a.version.values,index=a.study).to_dict()
lst = []
for k,v in tqdm(a_dict.items()):
    lst.append(df[(df['study']==k)&(df['version']==v)])
df = pd.concat(lst)

100%|██████████| 933/933 [00:03<00:00, 253.63it/s]


In [181]:
# studies not containing var_report files
print(len(list(set(dict_dates.keys()).difference(df['study'].unique()))))

44


In [196]:
df[['study','Last modified','version','file','link','comment']].sort_values(by='Last modified').to_csv('study_comments.tsv',sep='\t',index=False)